In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, LongType, DecimalType
)

In [0]:
load_path = '/Volumes/raw/full_load/full_load_csv/'
write_path = '/Volumes/raw/upsell/full_load/'

In [0]:
SCHEMA_CLIENTES = StructType([
    StructField("IdCliente", StringType(), True),
    StructField("FlEmail", LongType(), True),
    StructField("FlTwitch", LongType(), True),
    StructField("FlYouTube", LongType(), True),
    StructField("FlBlueSky", LongType(), True),
    StructField("FlInstagram", LongType(), True),
    StructField("QtdePontos", LongType(), True),
    StructField("DtCriacao", StringType(), True),
    StructField("DtAtualizacao", StringType(), True)
])

SCHEMA_TRANSACOES = StructType([
    StructField("IdTransacao", StringType(), True),
    StructField("IdCliente", StringType(), True),
    StructField("DtCriacao", StringType(), True),
    StructField("QtdePontos", LongType(), True),
    StructField("DescSistemaOrigem", StringType(), True)
])

SCHEMA_TRANSACAO_PRODUTO = StructType([
    StructField("IdTransacaoProduto", StringType(), True),
    StructField("IdTransacao", StringType(), True),
    StructField("IdProduto", StringType(), True),
    StructField("QtdeProduto", LongType(), True),
    StructField("VlProduto", DecimalType(12, 2), True)
])

In [0]:
from pyspark.sql.functions import col, substring, to_timestamp

# Leitura do arquivo csv clientes
schema_csv_clientes = "IdCliente string, FlEmail long, FlTwitch long, FlYouTube long, FlBlueSky long, FlInstagram long, QtdePontos long, DtCriacao timestamp, DtAtualizacao timestamp"
full_load_csv_clientes = spark.read.csv(
                                f"{load_path}clientes.csv",
                                sep=";",
                                header=True,
                                schema=SCHEMA_CLIENTES
)

# # Criar as colunas temporárias convertidas para timestamp
# full_load_csv_clientes = full_load_csv_clientes \
#     .withColumn("DtCriacaoTS", to_timestamp(substring(col("DtCriacao"), 1, 19), "yyyy-MM-dd HH:mm:ss")) \
#     .withColumn("DtAtualizacaoTS", to_timestamp(substring(col("DtAtualizacao"), 1, 19), "yyyy-MM-dd HH:mm:ss"))

# # Remover colunas originais e renomear as novas para os nomes originais
# full_load_csv_clientes = full_load_csv_clientes \
#     .drop("DtCriacao", "DtAtualizacao") \
#     .withColumnRenamed("DtCriacaoTS", "DtCriacao") \
#     .withColumnRenamed("DtAtualizacaoTS", "DtAtualizacao")

display(full_load_csv_clientes)

In [0]:
full_load_csv_clientes.printSchema()

In [0]:
# Salva em formato parquet
full_load_csv_clientes.write.mode("overwrite").parquet(
    f"{write_path}customers/"
)

In [0]:
spark.read.parquet("/Volumes/raw/upsell/full_load/customers/part-00000-tid-652902602118635973-2fdac786-73e3-425d-96f3-cdd4cdfb3f60-128-1.c000.snappy.parquet").printSchema()

In [0]:
spark.read.parquet("/Volumes/raw/upsell/cdc/customers/part-00000-83a31305-a855-4868-bd38-95a1e3963ecf-c000.snappy.parquet").printSchema()

In [0]:
# Leitura do arquivo csv produto
schema_csv_produtos = "IdProduto int, DescProduto string, DescCateogriaProduto string"
full_load_csv_produtos = spark.read.csv(
    f"{load_path}produtos.csv",
    sep=";",
    header=True,
    schema=schema_csv_produtos,
)
display(full_load_csv_produtos)

In [0]:
# Salva em formato parquet
full_load_csv_produtos.write.mode("overwrite").parquet(
    f"{write_path}products/"
)

In [0]:
# Leitura do arquivo csv transacao_produto
schema_csv_transacao_produto = 'IdTransacaoProduto string, IdTransacao string, IdProduto string, QtdeProduto long, VlProduto decimal(12,2)'
full_load_csv_transacao_produto = spark.read.csv(
    f"{load_path}transacao_produto.csv",
    sep=";",
    header=True,
    schema=SCHEMA_TRANSACAO_PRODUTO,
)
display(full_load_csv_transacao_produto)

In [0]:
# Salva em formato parquet
full_load_csv_transacao_produto.write.mode("overwrite").parquet(
    f"{write_path}transactions_product/"
)

In [0]:
# Leitura do arquivo csv transacoes
schema_csv_transacoes = 'IdTransacao string, IdCliente string, DtCriacao timestamp, QtdePontos long, DescSistemaOrigem string'
full_load_csv_transacoes = spark.read.csv(
    f"{load_path}transacoes.csv",
    sep=";",
    header=True,
    schema=SCHEMA_TRANSACOES
)

# # Criar coluna temporária convertida para timestamp
# full_load_csv_transacoes = full_load_csv_transacoes \
#     .withColumn("DtCriacaoTS", to_timestamp(substring(col("DtCriacao"), 1, 19), "yyyy-MM-dd HH:mm:ss"))

# # Remover coluna original e renomear a nova para nome original
# full_load_csv_transacoes = full_load_csv_transacoes \
#     .drop("DtCriacao") \
#     .withColumnRenamed("DtCriacaoTS", "DtCriacao")

display(full_load_csv_transacoes)

In [0]:
# Salva em formato parquet
full_load_csv_transacoes.write.mode("overwrite").parquet(
    f"{write_path}transactions/"
)

In [0]:
full_load_csv_transacoes.printSchema()

In [0]:
spark.read.parquet("/Volumes/raw/upsell/full_load/transactions/part-00000-tid-1616137082553178915-44be9ead-be84-4b31-b17d-52fe065bf42a-143-1.c000.snappy.parquet").printSchema()